In [1]:
# Import libraries

import numpy as np
!pip install lxml

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

In [2]:
#Create dataframe df0 consisting of three columns: PostalCode, Borough, and Neighborhood
ulr = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
tables = pd.read_html(ulr,header=0)
df0 = tables[0]
df0.tail()

,Postcode,Borough,Neighbourhood
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor
286,M9Z,Not assigned,Not assigned


In [3]:
#Ignore cells with a borough that is Not assigned
df0=df0[df0.Borough != 'Not assigned']
df1 = df0.sort_values(by=['Postcode','Borough'])
df1.reset_index(inplace=True)
df1.drop('index',axis=1,inplace=True)
df1.tail()

,Postcode,Borough,Neighbourhood
205,M9V,Etobicoke,Mount Olive
206,M9V,Etobicoke,Silverstone
207,M9V,Etobicoke,South Steeles
208,M9V,Etobicoke,Thistletown
209,M9W,Etobicoke,Northwest


In [4]:
#Merge rows with same Postcode into one row with the neighborhoods separated with a comma
df2 = df1.groupby(by=['Postcode','Borough']).agg(lambda x: ','.join(x))
df2.reset_index(level=['Postcode','Borough'], inplace=True)
df2.tail()

,Postcode,Borough,Neighbourhood
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie..."
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."
102,M9W,Etobicoke,Northwest


In [5]:
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
df2['Neighbourhood']=df2['Neighbourhood'].replace('Not assigned', df2['Borough'])
df2.tail()

,Postcode,Borough,Neighbourhood
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie..."
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."
102,M9W,Etobicoke,Northwest


In [6]:
df2.shape

(103, 3)